In [1]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
import torch
device = 0 if torch.cuda.is_available() else -1  # 0 = first GPU, -1 = CPU for speed
from arabert.preprocess import ArabertPreprocessor #preprocess text
from dotenv import load_dotenv
import re
from nltk.tokenize import sent_tokenize
import random
from transformers import pipeline
# from camel_tools.tokenizers.sent import simple_sentence_tokenize



load_dotenv()  # Loads HF_TOKEN automatically

True

In [2]:
#directly thro hugging face
from transformers import pipeline

# POS tagging
pos = pipeline(
    "token-classification",
    model="CAMeL-Lab/bert-base-arabic-camelbert-da-pos-msa",
    framework="pt",
    trust_remote_code=True  # VERY IMPORTANT
)

# NER
ner = pipeline(
    "ner",
    model="CAMeL-Lab/bert-base-arabic-camelbert-msa-ner",
    framework="pt",
    trust_remote_code=True  # VERY IMPORTANT
)


Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-da-pos-msa were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-msa-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained o

In [3]:
# genrating other MQS options
load_dotenv()  
from transformers import pipeline
from dotenv import load_dotenv
import os
import openai

# Load .env file
load_dotenv()

# Get the key
api_key = os.getenv("OPENAI_API_KEY")

# Debug check is it connecting?
if api_key:
    print("✅ OpenAI API key loaded successfully")
else:
    print("❌ Failed to load OpenAI API key. Check your .env file")

# Assign to OpenAI
openai.api_key = api_key


✅ OpenAI API key loaded successfully


In [4]:
import re

def arabic_sentence_split(text, max_words=150):
    # Split by sentence-ending punctuation first
    sentences = re.split(r'(?<=[\.\؟\!])\s+', text.strip())
    result = []
    for s in sentences:
        words = s.split()
        if len(words) <= max_words:
            result.append(s)
        else:
            # Chunk long sentences safely
            for i in range(0, len(words), max_words):
                result.append(" ".join(words[i:i+max_words]))
    return result


In [5]:
def generate_question_text(entity, pos, word, sentence=None):
    """Generate natural Arabic question from entity + POS with more coverage and variety."""
    
    # Person entity
    if entity == "PER":
        if pos in ["noun", "noun_prop"]:
            return f"من هو {word}؟"
        elif pos == "adj":
            return f"أي شخص وُصف بـ {word}؟"
        else:
            return f"إلى أي شخص تشير كلمة '{word}'؟"
    
    # Location 
    if entity == "LOC":
        return f"أين تقع {word}؟"
    
    # Organization
    if entity == "ORG":
        if pos == "noun":
            return f"ما هي المنظمة المسماة {word}؟"
        elif pos == "adj":
            return f"أي منظمة وُصفت بأنها {word}؟"
        else:
            return f"اذكر المنظمة المرتبطة بكلمة '{word}'؟"
    
    #Date / Time 
    if entity in ["DATE", "TIME"]:
        return f"متى حدث ذلك؟ (الإشارة إلى {word})"
    
    #Number / Quantity
    if entity in ["NUM", "QUANTITY", "PERCENT"]:
        return f"ما هي القيمة العددية المذكورة: ____ (الإجابة {word})؟"
    
    # Miscellaneous / Product / Event
    if entity in ["EVENT", "WORK_OF_ART", "MISC"]:
        return f"إلى أي شيء يشير {word}؟"
    
    # Default Cloze (fill-in-the-blank)
    if sentence:
        return f"أكمل الفراغ: {sentence.replace(word, '____')}"
    
    #Fallback in case
    return f"صف الكلمة: {word}"


In [6]:

import random

def generate_TF_question(sentence, entity_word, entity_type, ner_tags):
    """
    Generate a True/False question using GPT-3.5 to make the False statement more natural.

    sentence: the original sentence
    entity_word: the correct entity in the sentence
    entity_type: NER type, e.g., "LOC", "PER", "ORG"
    ner_tags: output from ner() for the text
    """
    # True statement
    true_statement = f"{sentence} (صح أم خطأ؟)"

    # Prepare prompt for GPT-3.5
    prompt = f"اجعل الجملة التالية خاطئة مع الحفاظ على المعنى العام والموضوع: {sentence}"

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "أنت مساعد لإنشاء أسئلة صح أو خطأ باللغة العربية."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.7
        )

        # Extract generated text
        false_sentence = response['choices'][0]['message']['content'].strip()
        false_statement = false_sentence + " (صح أم خطأ؟)"

    except Exception as e:
        print("Error generating false statement:", e)
        # fallback: swap entity as before
        other_entities = [tag['word'] for tag in ner_tags 
                          if tag['entity'] == entity_type and tag['word'] != entity_word]
        distractor = random.choice(other_entities) if other_entities else entity_word + "X"
        false_statement = sentence.replace(entity_word, distractor) + " (صح أم خطأ؟)"

    # Randomly choose True or False
    if random.choice([True, False]):
        return {"type": "TF", "statement": true_statement, "answer": True}
    else:
        return {"type": "TF", "statement": false_statement, "answer": False}


In [7]:
difficulty_settings = {
    "easy": {
        "num_questions": 5,
        "tf_ratio": 0.7,       
        "mcq_distractor_type": "simple"  
    },
    "medium": {
        "num_questions": 10,
        "tf_ratio": 0.5,       
        "mcq_distractor_type": "medium"  
    },
    "hard": {
        "num_questions": 15,
        "tf_ratio": 0.3,       
        "mcq_distractor_type": "challenging"  
    }
}


In [8]:
import random
import openai

def make_mq_options(entity_label, correct_entity, sentence, num_distractors=2):
    distractors = []

    prompt = (
        f"أعطني {num_distractors} كلمات عربية (كلمة واحدة لكل بديل) "
        f"تكون بدائل خاطئة لكلمة '{correct_entity}' في الجملة التالية: {sentence}. "
        f"كن محترمًا وابتعد عن أي كلمات جارحة أو مسيئة أو عنصرية أو جنسية. "
        f"أجب فقط بالكلمات مفصولة بفاصلة."
    )

    gpt_failed = False

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "أنت مساعد لإنشاء خيارات متعددة الاختيار باللغة العربية."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0.7
        )

        generated_text = response['choices'][0]['message']['content'].strip()

        # Extract distractors
        for option in generated_text.split("،"):
            option_clean = option.strip()
            if option_clean and option_clean != correct_entity:
                distractors.append(option_clean)

    except Exception as e:
        print("Error generating distractors:", e)
        gpt_failed = True

    # Only use fallback if GPT failed completely
    if gpt_failed:
        distractors = [f"خيار{i+1}" for i in range(num_distractors)]

    # Limit to requested number of distractors
    distractors = distractors[:num_distractors]

    # Combine correct answer + distractors and shuffle
    options = [correct_entity] + distractors
    random.shuffle(options)

    return options


In [9]:
def generate_MCQ_question(entity_word, sentence, entity_type="LOC", full_text=None):
    """
    Generate MCQ using entity as correct answer and GPT-3.5-generated distractors.
    """
    # Generate the question text
    question_text = generate_question_text(entity_type, "noun", entity_word, sentence)
    
    # Generate distractors using GPT-3.5
    options = make_mq_options(entity_type, entity_word, sentence, num_distractors=2)
    
    return {"type": "MCQ", "question": question_text, "options": options, "answer": entity_word}


In [10]:
from tqdm import tqdm
import pyarabic.araby as araby

def make_quiz(text, level="medium"):
    settings = difficulty_settings.get(level, difficulty_settings["medium"])
    num_questions = settings["num_questions"]
    tf_ratio = settings["tf_ratio"]

    questions = []
    seen = set()
    sentences = arabic_sentence_split(text, max_words=200)

    # Batch NER + POS once since camel bert only takes 512 token at once
    ner_tags_all = ner(sentences, batch_size=32)
    pos_tags_all = pos(sentences, batch_size=32)

    for i, sentence in enumerate(tqdm(sentences)):
        if len(questions) >= num_questions:
            break
        ner_tags = ner_tags_all[i]
        pos_tags = pos_tags_all[i]
        if not ner_tags:
            continue

        entity = ner_tags[0]
        word = entity['word']
        entity_type = entity['entity']
        pos_tag = next((p['entity'] for p in pos_tags if p['word'] == word), "noun")

        # TF vs MCQ decision
        if random.random() < tf_ratio:
            q = generate_TF_question(sentence, word, entity_type, ner_tags)
        else:
            q = generate_MCQ_question(word, sentence, entity_type, full_text=text)

        # ensure no repeats
        sig = (q["type"], q.get("question"), q.get("answer"))
        if sig not in seen:
            questions.append(q)
            seen.add(sig)

    return questions


In [11]:
import re

"""
Pre-process Arabic text while keeping relevant content for NLP:

- Skips empty lines
- Skips page numbers (English 1, 2… and Arabic-Indic ١, ٢…)
- Skips figure/table captions starting with شكل / جدول
- Removes inline references [1], (2) etc. (English + Arabic-Indic)
- Stops processing at references section
- Skips initial metadata/intros until first Arabic paragraph
- Skips first irrelevant pages in textbooks
- force arabic parsing back as it messed up the reading,tho it messes the logic compariosn in models above
"""

def is_textbook_front(line):
    """Return True if the line is likely textbook front matter."""
    keywords = ["الطبعة", "المؤلف", "المحرر", "الحقوق محفوظة", "نسخة", "جامعة", "الكتاب", "المقدمة"]
    return any(k in line for k in keywords)

    
def clean_arabic_text(text):
    lines = text.splitlines()
    cleaned_lines = []

    skip_intro = True
    arabic_re = re.compile(r'[\u0600-\u06FF]')
    
    for line in lines:
        line = line.strip()
        if not line:
            continue

        if skip_intro:
            if is_textbook_front(line) or (len(arabic_re.findall(line)) < 5):
                continue
            else:
                skip_intro = False

        # Skip page numbers (English or Arabic-Indic)
        if re.match(r'^([0-9]+|[٠-٩]+)$', line):
            continue

        # Skip figure/table captions
        if re.match(r'^(شكل|جدول)\s*([0-9]+|[٠-٩]+)', line):
            continue

        # Stop at references section
        if line.startswith(("المراجع", "قائمة المراجع")):
            break

        # Remove inline references like [1], (2), [١], (٢)
        line = re.sub(r'\[[0-9٠-٩]+\]|\([0-9٠-٩]+\)', '', line)

        cleaned_lines.append(line)

    # Join all cleaned lines
    cleaned_text = "\n".join(cleaned_lines)

   

    return cleaned_text


In [12]:
# Strategy for 30 max we do less pages we will need to chunck the request to model 
from concurrent.futures import ThreadPoolExecutor
#the cleaning causes issues sometimes in 1 or 2 question idk why ;check later
def make_quiz_from_large_text(text, level="medium"):
    cleaned_text=clean_arabic_text(text)
    sentences =arabic_sentence_split(cleaned_text, max_words=200)
    chunks = [sentences[i:i+200] for i in range(0, len(sentences), 200)]
    results = []

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(make_quiz, " ".join(chunk), level) for chunk in chunks]
        for f in futures:
            results.extend(f.result())

    return results



### Pre-trained Model Evaluation

The pre-trained CAMeLBERT model demonstrates strong performance on **MSA Arabic datasets**, providing a reliable baseline for quiz generation. According to the original paper [1] , the reported performance is:

| Task | Dataset | Variant | Performance (MSA) |
|------|---------|--------|-----------------|
| NER  | ANERcorp | MSA   | 82.4 % |
| POS  | PATB     | MSA   | 98.3 % |

These results indicate that the model achieves high accuracy on both **named entity recognition** and **part-of-speech tagging** for Modern Standard Arabic.

---

### Domain-Specific Evaluation

We then evaluated the pre-trained model on **our domain data** Arabic historical, literary, and theoretical texts, which are non-numeric and span a broad range of topics. 

Initially, **human evaluation** was used to assess quiz quality, and the results are summarized below. In the next step, we will evaluate the **labeled domain data** to compute quantitative performance scores for the pre-trained model on our specific dataset.


# Quiz Human Evaluation

The evaluation was performed using the **Quiz NLP-Generated Evaluation Metric**, which relies on **human evaluation** and is considered the gold standard for assessing subjective tasks such as quiz generation. Following a methodology similar to Callison-Burch [2], who demonstrated that aggregated non-expert human judgments can reliably reflect quality in natural language tasks, undergraduate students assessed the model-generated quizzes on multiple dimensions:

### Evaluation Dimensions

| Dimension          | Meaning                                                          |
| ------------------ | ---------------------------------------------------------------- |
| Correctness        | Is the blank based on a real entity or part-of-speech (POS)?     |
| Clarity            | Is the question understandable?                                  |
| Context coherence  | Does the sentence remain meaningful after removing the word?     |
| Difficulty         | Does the question match the intended level (easy, medium, hard)? |
| Distractor quality | (For multiple-choice questions) Are wrong options plausible?     |

**Scoring Method:**
Each question is rated from **1 to 5** on each dimension they add +1 for each question. The average across dimensions gives a **Quality Score** for each question. Feedback was used to refine the tokenizer and distractor generation for future iterations.

### Human Evaluation Results

| Evaluator       | Quality Score /5 | Comments                                                                                           | Level  |
| --------------- | ---------------- | -------------------------------------------------------------------------------------------------- | ------ |
| Norah Alrajhi   | 5                | Clear, but some questions were long → fixed using tokenizer after feedback                         | Hard   |
| Letifah Aladaad | 3                | Clarity issues: some grammar off in Arabic; we parse exact text as authored, no grammar correction | Medium |
| Norah Alqahtani | 4                | Some questions slightly ambiguous; overall acceptable                                              | Easy   |
| Fajir Alkethiri | 4                | Medium-level questions mostly fine but distractors sometimes are wired                                                                | Medium |
| Ghala Alqahtani | 3                | Some questions too difficult for medium level; distractors sometimes too obvious                   | Medium |

**Notes:**

* Feedback helped improve question clarity and distractor plausibility by optimizing the GPT prompt.
* Difficulty levels were verified against the intended educational level found. When sending level, it was always medium. I found a bug that levels picked in Arabic never mapped to the actual code levels, and fixed that.
* Overall, the human evaluation confirms that the model generates quizzes of reasonable quality across the target domains.

### Labeled Domain Data Evaluation

In [13]:
#Sentences only, raw text from the selected category (history or literature).
import wikipediaapi
import random
from nltk.tokenize import sent_tokenize

wiki = wikipediaapi.Wikipedia(language='ar', extract_format=wikipediaapi.ExtractFormat.WIKI,
                              user_agent='my-domain-eval-script')

def get_sentences_from_category(category_name, n_sentences=50):
    cat = wiki.page(f"تصنيف:{category_name}")
    members = list(cat.categorymembers.values())
    sentences = []

    for page in members:
        text = page.text
        if text:
            sents = sent_tokenize(text)
            sentences.extend(sents)
        if len(sentences) >= n_sentences * 2:  # collect extra to sample
            break
    return random.sample(sentences, min(n_sentences, len(sentences)))

# Example: 50 history sentences, 50 literature sentences
history_sents = get_sentences_from_category("تاريخ", n_sentences=50)
literature_sents = get_sentences_from_category("أدب", n_sentences=50)

domain_sentences = history_sents + literature_sents
print(f"Total sentences: {len(domain_sentences)}")


Total sentences: 100


In [14]:
#Label them with
ner_results = [ner(sentence) for sentence in domain_sentences]
pos_results = [pos(sentence) for sentence in domain_sentences]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
#export predicted into csv and ill edit it maunaly if found any mistake then we will have 100 sent labaled 
# run once
# import pandas as pd

# rows = []
# for sentence, ner_sent, pos_sent in zip(domain_sentences, ner_results, pos_results):
#     for t_ner, t_pos in zip(ner_sent, pos_sent):
#         rows.append({
#             "sentence": sentence,
#             "token": t_ner["word"],
#             "predicted_POS": t_pos["entity"],
#             "predicted_NER": t_ner["entity"]
#         })

# df = pd.DataFrame(rows)
# df.to_csv("domain_predicted_labels.csv", index=False, encoding="utf-8-sig")


In [17]:
# After manual editing of labels we evaluate F1 now

import pandas as pd
from sklearn.metrics import classification_report

# Load edited file
df = pd.read_csv("domain_predicted_labels.csv")  # your manually fixed labels

# Store true labels from the file
true_pos = df['predicted_POS'].tolist()
true_ner = df['predicted_NER'].tolist()

# Store model predictions
pred_pos = []
pred_ner = []

# Iterate through each token and get predictions: as in sheet sent and token
for sentence, token in zip(df['sentence'], df['token']):
    
    # --- POS prediction ---
    pos_results = pos(sentence)
    # Hugging Face returns list of dicts: {'word': ..., 'entity': ..., 'score': ...}
    pos_tokens = [r['word'] for r in pos_results]
    pos_tags = [r['entity'] for r in pos_results]

    # --- NER prediction ---
    ner_results = ner(sentence)
    ner_tokens = [r['word'] for r in ner_results]
    ner_tags = [r['entity'] for r in ner_results]

    # --- Align by token robustly ---
    # Hugging Face tokenization may split Arabic words into subwords (##)

    # POS alignment
    matched_pos = None
    for idx, w in enumerate(pos_tokens):
        if token.startswith(w.replace("##","")):  # match token start
            matched_pos = pos_tags[idx]
            break

    # NER alignment
    matched_ner = None
    for idx, w in enumerate(ner_tokens):
        if token.startswith(w.replace("##","")):  # match token start
            matched_ner = ner_tags[idx]
            break

    # fallback if not found
    if matched_pos is None:
        matched_pos = "O"
    if matched_ner is None:
        matched_ner = "O"

    pred_pos.append(matched_pos)
    pred_ner.append(matched_ner)

# Compute F1 for our domain data (50 rows of history and literature data)
print("POS Classification Report:\n")
print(classification_report(true_pos, pred_pos))

print("NER Classification Report:\n")
print(classification_report(true_ner, pred_ner))


POS Classification Report:

              precision    recall  f1-score   support

         adj       0.00      0.00      0.00         1
         adp       0.00      0.00      0.00         1
    conj_sub       0.00      0.00      0.00         1
        noun       0.78      0.25      0.38        28
   noun_prop       0.43      0.94      0.59        17
       propn       0.00      0.00      0.00         1
        verb       0.00      0.00      0.00         0

    accuracy                           0.47        49
   macro avg       0.17      0.17      0.14        49
weighted avg       0.59      0.47      0.42        49

NER Classification Report:

              precision    recall  f1-score   support

       B-LOC       1.00      1.00      1.00         6
      B-MISC       1.00      1.00      1.00         6
      B-PERS       1.00      1.00      1.00        18
      I-MISC       1.00      1.00      1.00         1
       I-ORG       1.00      1.00      1.00         1
      I-PERS       1.0

C:\Users\renad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\renad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\renad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

**Reference**

[1] G. Inoue, Bashar Alhafni, Nurpeiis Baimukan, Houda Bouamor, and N. Habash, “The Interplay of Variant, Size, and Task Type in Arabic Pre-trained Language Models.,” pp. 92–104, Apr. 2021.

‌[2] C. Callison-Burch, "Fast, cheap, and creative: Evaluating translation quality using Amazon's Mechanical Turk," in *Proc. Conf. Empirical Methods in Natural Language Processing (EMNLP)*, Singapore, 2009, pp. 286–295. [Online]. Available: https://aclanthology.org/D09-1030/

